# Building ion current models from data 

In this exercise we will explore some of the challenges that come with optimizing a model to data. We'll use a moderately complex model for the ultra-rapidly activating delayed rectifier current ($I_{\rm{Kur}}$). This model is simple in the sense that it has identical rate constants for each activating and deactivating transition, and it is linear. However, you'll see that there are still 12 parameters that can be varied during the fitting process, which can make for challenges in hand tuning the model to a simple activation curve.

We are using a 5-state Markov model published by Zhou et al. 2012:

<img src="fig/Model.svg" width=400>

Where: $$\alpha{} = e^{\frac{V-P1}{P2}}$$.
$$\beta{} = \frac{e^{\frac{V-P3}{P4}}\cdot{}e^{\frac{-(V-P5)}{P6}}}{P7+P8\cdot{}e^{\frac{-(V-P9)}{P10}}}$$.
$$ K_1 = const_1 $$
$$ K_2 = const_2 $$

Below we will take some time to try to fit this model just to some activation data for this current. In this case we have also allowed the peak conductance ($g_K$) to be a free parameter for your tuning. See how close you can get to the experimental data.

## Exercise 1: Fitting ionic model parameters
### Hand tuning model parameters

In [ ]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import scipy.optimize as opt
import math

In [ ]:
from ipywidgets import interact, FloatSlider
from widget.ObFunc import f, Activation
import widget.K_widget as K
import importlib

In [ ]:
importlib.reload(K)
K.Markov_Widget()

You can probably see that this model is underdetermined and that there are far more parameters than necessary or helpful for fitting this activation curve. What do you think is the reason for this?

### Algorithmic optimization to find parameters
Next we will employ a commonly-used algorithm (the Nelder-Mead simplex, or AMOEBA) to optimize the parameter set, and see if it can beat you for accuracy. This algorithm is a form of derivative-free approach to optimization our voltage clamp activation function: 

First let's specify the experimental data you will use for fitting:

In [ ]:
SSA_data = np.loadtxt("widget/SS.txt",dtype='float')

V = SSA_data[:,0] # voltage
I = SSA_data[:,1] # current

Next we have to define a cost function for comparing the model generated data and the experimental data we have imported.

In [ ]:
def cost(P,V,I,duration):
    
    P = P.tolist() # Opt.fmin returns an array every iteration...
                    # but we pass a list to Activation, so we need to typeset.
    outs = Activation(P,V,duration)
    model_peaks = outs['I_peak']
       
    dev_vector = model_peaks - I
    error = np.linalg.norm(dev_vector,2) # calculating the error as the L2 norm
    
    return error

Run the optimization and see how it performs:

In [ ]:
step_length = 1000
default_params = [45,20,65,50,20,15,1,0.02,29,15,1e-5,1e-5,0.4]
init_params = [...]# insert your parameter set as a starting point

[P_opt, f_opt, iters, funcalls, warnflag] = opt.fmin(cost, init_params, args=(V,I,step_length), maxiter = 200, maxfun = 200, full_output=True, disp=True)
P_opt = P_opt.tolist()

Finally plot your optimized data compared to the experimental data below. Did the algorithm do better than your hand tuning?

In [ ]:
opt_out = Activation(P_opt,V,step_length)
opt_model_I = opt_out['I_peak']
hand_out = Activation(init_params,V,step_length)
hand_model_I = hand_out['I_peak']
default_out = Activation(default_params,V,step_length)
default_model_I = default_out['I_peak']

plt.figure()
plt.plot(V,I,'b-')
plt.plot(V,opt_model_I,'r-')
plt.plot(V,hand_model_I,'k-')
plt.plot(V,default_model_I,'g-')
plt.xlabel('Step potential [mV]')
plt.ylabel('Current (A/F)')
plt.legend(['Expirement','Optimized','Hand-tuned','Default'])

print(f'Cost for default parameters = {cost(np.array(default_params),V,I,step_length):.5f}')
print(f'Cost for manually trained parameters = {cost(np.array(init_params),V,I,step_length):.5f}')
print(f'Cost for optimized parameters = {f_opt:.5f}')

<a id="2"></a>
# Combining ion current models in the action potential 

Next we will explore how the major cardiac currents contribute to differences in the atrial and ventricular action potentials and hoping to achieve some intuitive understanding of what the defining currents are in each of these tissues. 

First we need to get a little more familiar with what one of these models looks like. The process for constructing an action potential model from the many ion channels, tranporters, and buffer models that make a cell, is an extension of what you have already learned from the earlier exercises.

You need:
1. Your system of equations (from which you formulate the RHS)
2. An initial condition (starting values for each variable in the RHS)
3. The collection of parameters (generally constants) necessary to calculate the RHS 
4. A time vector over which to integrate the system

## Exercise 2: the Hodgkin-Huxley model

We'll look first at the Hodgkin-Huxley model. It has an activation gated ($n$) ${\rm K}^+$  current, a ${\rm Na}^+$ current with both activation ($m$) and inactivation ($h$) gates, a small nondescript leak current ($I_{\rm L}$) to maintain resting potential, and it accepts a stimulus current $I_{\rm app}$:
$$C_{\rm m}\frac{{\rm d}V}{{\rm d}t} = -(g_{\rm Na} m^3h(V-E_{\rm Na}) + g_{\rm K} n^4(V-E_{\rm K}) + g_{\rm L}(V-E_{\rm L}) + I_{\rm app}).$$

where $m$ is controlled by:
$$ \frac{{\rm d}m}{{\rm d}t} = \alpha_m (1-m)-\beta_m m$$

$h$ is controlled by:
$$ \frac{{\rm d}h}{{\rm d}t} = \alpha_h (1-h)-\beta_h h$$

and $n$ is controlled by:
$$ \frac{{\rm d}n}{{\rm d}t} = \alpha_n (1-n)-\beta_n n$$

First some tools and then we start with the right hand side:

In [ ]:
# Import necessary packages
import numpy as np
from math import exp 
import matplotlib.pylab as plt
%matplotlib inline
from scipy.integrate import odeint

In [ ]:
def rhs(y,t,p):
    
    # unpack the solution vector
    m, h, n, V = y 
    
    # unpack the passed parameters
    Cm = p['Cm']
    E_Na = p['E_Na']
    E_K = p['E_K']
    E_L = p['E_L']
    gNa = p['gNa']
    gK = p['gK']
    gL = p['gL']
    I_amp = p['I_amp']
    
    # calculate the transition rates
    alpha_m = 0.1*(V+40.0)/(1.0-np.exp(-(V+40.0)/10.0))
    beta_m  = 4.0*np.exp(-(V+65.0)/18.0)
    alpha_h = 0.07*np.exp(-(V+65.0)/20.0)
    beta_h  = 1.0/(1.0+np.exp(-(V+35.0)/10.0))
    alpha_n = 0.01*(V+55.0)/(1.0-np.exp(-(V+55.0)/10.0))
    beta_n  = 0.125*np.exp(-(V+65)/80.0)

    # calculate the currents
    I_app = -I_amp if 2<t<4 else 0.0
    I_Na = gNa*m**3*h*(V - E_Na)
    I_K = gK*n**4*(V - E_K)
    I_L = gL*(V - E_L)
    
    # calculate the derivatives
    dm_dt = alpha_m*(1-m) - beta_m*(m)
    dh_dt = alpha_h*(1-h) - beta_h*(h)
    dn_dt = alpha_n*(1-n) - beta_n*(n)
    dV_dt = -(I_Na + I_app + I_K)/Cm
    
    # assemble and return dy/dt
    return[dm_dt,dh_dt,dn_dt,dV_dt]

In [ ]:
#Initial conditions   
m0 = 0
h0 = 1
n0 = 0
V0 = -80
y0 = [m0,h0,n0,V0]

#Parameters
p = {}
p['Cm'] = 1.0 # uF/cm^2
p['E_Na'] = 50 # mV
p['E_K'] = -80 # mV
p['E_L'] = -75 # mV
p['gNa'] = 120 # mS/cm^2
p['gK'] = 40 # mS/cm^2
p['gL'] = 0.3 # mS/cm^2
p['I_amp'] = 100 # uA/cm^2

#The time vector
time = np.linspace(0,20,1000)

In [ ]:
# Run the model
# Your code here:
y = odeint(...)

m = y[:,0]
h = y[:,1]
n = y[:,2]
V = y[:,3]

#Plot the voltage
plt.subplot(1,2,1)
plt.plot(...)
#plot the state variables
plt.subplot(1,2,2)
plt.plot(...)

Note that the length of the action potential.  Is this more consistent with a cardiac action potential or a neuronal action potential. Also recall Dr. McCulloch's description of the experimental system used to develop the model.

## Exercise 3: A more complex model...

Next we'll work with a human ventricular myocyte model. This is one of several human-specific ventricular myocyte models, and was developed by Eleonora Grandi at UC Davis. The code we will run is slightly simplified from the published version, but still quite complex. It is generally difficult to interpret any new model, but getting used to looking for the necessary elements is the best way to start. Let's take a look:

* [Grandi-Bers Ventricular RHS](widget/GBV_RHS.py)
* [Grandi-Bers Ventricular Initialization](widget/GBV_D.py)
* [Grandi-Bers Ventricular Run](widget/GBV_run.py)


Our objective with this section is to develop some familiarity with how these cell models are structured, and some intuition for how the major currents in the ventricle and atria determine the integrated electrophysiology of the cell. Unfortunately we won't have a chance to look at complex effects, such as frequency-dependent outcomes, but we can at least gain a sense of how each current contributes to changes in action potential morphology.  

First, some more tools.

In [ ]:
from ipywidgets import interact, FloatSlider, Dropdown
import widget.L6_widgets as L6
import importlib

Now we'll load a widget that will let you inspect the behaviour of the ventricular AP model by changing the conductances of all the major sarcolemmal currents. 

*FIRST*: Answer some intuitive questions that you should already have a good idea about:

1. What is the maximum reachable positive potential?
2. Which current maintains a stable resting potential?
3. Is it possible to make the human AP as short as a mouse AP (~30 ms)? How?
4. Is it possible to prolong the AP by increasing a potassium channel conductance?
5. Is it possible to increase the peak of the calcium transient by increasing a potassium channel conductance?
6. How do INCX and INKa modulate repolarisation?
7. What are the background currents (NaB, ClB, CaB)? Are they important?

**See if the model agrees...**

In [ ]:
importlib.reload(L6)
L6.VentricularAPWidget()

## Exercise 4: Reachable model configurations

Because of the uncertainty in model construction, and variability in the underlying experimental data, it is always tempting to ask what are the major differences between models thought to represent different biological contexts. The next widget asks you to try to turn a ventricular cell into an atrial cell. We have made it somewhat easier by using an atrial model from the same authors. See how close you can get.

In [ ]:
importlib.reload(L6)
L6.VentricularAPWidget2()

### What about in the other direction?

Is it possible to make the atrial model look like a ventricular cell?

In [ ]:
importlib.reload(L6)
L6.AtrialAPWidget()